## Settings
<a name="TOC"></a>
1. [PEC](#1)

2. [Total energy deviation, coherences, populations](#2)

3. [Ehrenfest positions](#3)

4. [decoherence time](#4)

In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib
import matplotlib.font_manager as fm
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt   # plots
import itertools
import numpy as np
import time
import warnings

from liblibra_core import *
import util.libutil as comn
from libra_py import units
import libra_py.models.Holstein as Holstein
import libra_py.models.Tully as Tully
import libra_py.models.Subotnik as Subotnik
import libra_py.models.Esch_Levine as Esch_Levine
from libra_py import dynamics_plotting

import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

#from recipes import shxf, mqcxf, ehxf
#from recipes import ehrenfest_adi_ld, ehrenfest_adi_nac, ehrenfest_dia, mfsd
#from recipes import fssh, sdm, bcsh

import libra_py.dynamics.exact.compute as dvr
import libra_py.dynamics.exact.save as dvr_save

#from matplotlib.mlab import griddata
#%matplotlib inline 
warnings.filterwarnings('ignore')

## Color code
colors = {}
colors.update({"11": "#D20C0C", "12": "#FF5339", "13": "#FF8965"}) #red
colors.update({"21": "#00790C", "22": "#49A93F", "23": "#7DDB6D"}) #green
colors.update({"31": "#0009FF", "32": "#7145FF", "33": "#B176FF"}) #blue
colors.update({"41": "#FFA100", "42": "#FFD249", "43": "#FFFF7C"}) #orange
colors.update({"51": "#FF1DCE", "52": "#FFCDFF"}) #magenta
colors.update({"61": "#00ffea", "62": "#70FFFF", "63": "#AFFFFF"}) #cyan
colors.update({"71": "#bd20ff", "72": "#F461FF", "73": "#FF97FF"}) #indigo

clrs_index = ["11", "21", "31", "41", "51", "61", "71", "12", "22", "32", "42", "52", "62", "72" "13","23", "33", "53", "73"]


## Fig.1 Potential energy curves
<a name="1"></a>[Back to TOC](#TOC)

In [ ]:
def compute_model(q, params, full_id):

    model = params["model"]
    res = None
    
    if model==1:        
        res = Holstein.Holstein2(q, params, full_id) 
    elif model==2:
        res = Tully.Tully3(q, params, full_id)
    elif model==3:
        res = Subotnik.double_arch_geometry(q, params, full_id)
    elif model==4:
        res = Esch_Levine.JCP_2020(q, params, full_id)
    else:
        pass            

    return res

In [ ]:
model_params1 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.000}
model_params2 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.001}
#model_params3 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.01}
model_params3 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.05}
model_params4 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0, -0.01], "x_n":[0.0,  0.5],"k_n":[0.002, 0.008],"V":0.001}

model_params5 = {"model":2, "model0":2, "nstates":2} # ECR
model_params6 = {"model":3, "model0":3, "nstates":2} # DAG

model_params7 = {"model":4, "model0":4, "nstates":2, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.0, "i_crit":2, "delta":0.01 } # Esch-Levine

model_params8 = {"model":4, "model0":4, "nstates":3, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.0, "i_crit":3, "delta":0.01 } # Esch-Levine

model_params9 = {"model":4, "model0":4, "nstates":5, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.0, "i_crit":4, "delta":0.01 } # Esch-Levine

model_params10 = {"model":4, "model0":4, "nstates":5, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.02, "i_crit":3, "delta":0.01 } # Esch-Levine

all_model_params = [model_params1, model_params2, model_params3, model_params4, 
                    model_params5, model_params6, 
                    model_params7, model_params8, model_params9, model_params10
                   ]

In [ ]:
# 0 - Holstein, trivial crossing, 2 level
# 1 - Holstein, strong nonadiabatic, 2 level
# 2 - Holstein, adiabatic, 2 level
# 3 - Holstein, double crossing, strong nonadiabatic, 2 level
# 4 - Tully, extended crossing with reflection, 2 level
# 5 - Double arch geometry or symmetrized ECWR, 2 level
# 6 - Esch-Levine, LZ-like, 2 level
# 7 - Esch-Levine, 1 crosses 2 parallel, 3 level
# 8 - Esch-Levine, 1 crosses 4 evenly-spaced parallel, 5 level
# 9 - Esch-Levine, 1 crosses 4 parallel split into 2 groups, 5 level

#################################
# Give the model used an index
model_indx = 4
################################

model_params = all_model_params[model_indx]

In [ ]:
def flip(items, ncol):
    return itertools.chain(*[items[i::ncol] for i in range(ncol)])

In [ ]:
def plot_pec(_compute_model, model_params, xmin, xmax, dx, states_of_interest=[0,1], _ndof=1, _active_dof=0, _all_coordinates=[0.0], 
             show_nac_abs=0, frac_nac=1, _legend_ncol=1, _legend_font=30,  _loc="best", _bbox_to_anchor=""):
    X = []
    nsteps = int((xmax - xmin) / dx) + 1

    for i in range(nsteps):
        X.append(xmin + i * dx)
    comn.check_input(model_params, {}, ["nstates"])
    n = model_params["nstates"]
    nstates = n
    
    ham = nHamiltonian(nstates, nstates, _ndof) # ndia, nadi, nnucl
    ham.init_all(2)
    
    hdia, hadi, nac, nac_abs  = [], [], [], []
    uij = []              # projecitions of the MOs onto elementary basis
    
    for k1 in range(nstates):
        hadi.append([])
        hdia.append([])
        nac_k1 = []
        nac_abs_k1 = []
        uij_k1 = []
        for k2 in range(nstates):
            uij_k1.append([])
            nac_k1.append([])
            nac_abs_k1.append([])
        uij.append(uij_k1)
        nac.append(nac_k1)
        nac_abs.append(nac_abs_k1)
    
    for i in range(nsteps):

        scan_coord = MATRIX(_ndof, 1);
        for j in range(_ndof):
            scan_coord.set(j, 0, _all_coordinates[j])
        scan_coord.set(_active_dof, 0, X[i])

        # Diabatic properties
        ham.compute_diabatic(_compute_model, scan_coord, model_params)

        # Adiabatic properties
        ham.compute_adiabatic(1);

        U = ham.get_basis_transform()
        #P = U * U.H()  # population matrix

        for k1 in range(nstates):
            hadi[k1].append(ham.get_ham_adi().get(k1, k1).real)
            hdia[k1].append(ham.get_ham_dia().get(k1, k1).real)

            for k2 in range(nstates):
                uij[k1][k2].append(U.get(k1,k2).real**2 + U.get(k1,k2).imag**2)
                nac_k1_k2 = ham.get_dc1_adi(0).get(k1, k2).real
                nac[k1][k2].append(nac_k1_k2)
                nac_abs[k1][k2].append( abs(nac_k1_k2) )
    
    fig, ax1 = plt.subplots(figsize=(11.5,6), dpi=300)
    ax1.tick_params(axis='both', labelsize=30)
    
    ax1.set_xlabel('$R$ (Bohr)', fontsize=30)
    #ax1.set_ylabel('energy (a.u.)', labelpad=15, fontsize=30)
    ax1.set_ylabel('energy (Ha)', fontsize=30)
    ax1.margins(x=0)

    for k1 in states_of_interest:
        #plt.plot(X, hdia[k1], label='$H_{%i%i}$' % (k1,k1), linewidth=7, ls="--", color = colors[clrs_index[k1]])
        ax1.plot(X, hadi[k1], label='$E_{%i}$' % (k1), linewidth=5, color = colors[clrs_index[k1]])
        
    if(frac_nac == 1):
        txt_f=""
    else:
        txt_f="/" + F"${frac_nac}$"
        
    cnt = n
    for k1 in states_of_interest:
        for k2 in states_of_interest:
            if k2>k1:
                if show_nac_abs:
                    ax1.plot(X, [x/frac_nac for x in nac_abs[k1][k2]], label='$|d_{%i%i}|$' % (k1,k2) +txt_f, linewidth=5, ls="--", color = colors[clrs_index[cnt]], zorder=-1)
                else:
                    ax1.plot(X, [x/frac_nac for x in nac[k1][k2]], label='$d_{%i%i}$' % (k1,k2) +txt_f, linewidth=5, ls="--", color = colors[clrs_index[cnt]], zorder=-1)
                cnt = cnt + 1
    
    handles, labels = ax1.get_legend_handles_labels()
    
    if _bbox_to_anchor== "":
        ax1.legend(flip(handles, _legend_ncol), flip(labels, _legend_ncol), fontsize=_legend_font, frameon=False, loc = _loc,  ncol=_legend_ncol,
                columnspacing=0.5, handlelength=2.5)
    else:
        ax1.legend(flip(handles, _legend_ncol), flip(labels, _legend_ncol), fontsize=_legend_font, frameon=False, loc = _loc,  ncol=_legend_ncol,
                bbox_to_anchor=_bbox_to_anchor, columnspacing=0.5, handlelength=2.5)
    #ax1.legend(fontsize=_legend_font, frameon=False, ncol=_legend_ncol)
    

In [ ]:
list_states = [x for x in range(model_params["nstates"])]

plot_pec(compute_model, all_model_params[4], -15, 15, 0.05, show_nac_abs=1, frac_nac=1, _loc="lower left") #ECR
plot_pec(compute_model, all_model_params[5], -15, 15, 0.05, show_nac_abs=1, frac_nac=1, _loc="lower right") #DAG
plot_pec(compute_model, all_model_params[1], -4.0, 5.0, 0.05, show_nac_abs=1, frac_nac=20) #Holstein
plot_pec(compute_model, all_model_params[3], -4.0, 5.0, 0.05, show_nac_abs=1, frac_nac=20) #Holstein
plot_pec(compute_model, all_model_params[7], -6.0, 6.0, 0.05, states_of_interest=[0,1,2], show_nac_abs=1, frac_nac=20, _legend_ncol=3, _legend_font=27) #Esch-Levine

In [ ]:
plot_pec(compute_model, all_model_params[1], -4.0, 5.0, 0.05, show_nac_abs=1, frac_nac=20, _loc="upper right", _bbox_to_anchor=(1.03, 1)) #Holstein

## Fig. 2, 3, 4 | Total energy deviation, coherence, population
<a name="2"></a>[Back to TOC](#TOC)

In [ ]:
def compute_coherence(file):
    with h5py.File(file, "r") as f:
        C_ADI = f["Cadi/data"][:,:,:]

    nts, ntraj, nst = C_ADI.shape

    # Reshape C_ADI for better performance
    C_ADI_reshaped = C_ADI.reshape(nts, ntraj, nst, 1)

    # Compute rho2 using vectorized operations
    rho2 = np.sum(np.abs(C_ADI_reshaped) ** 2 * np.abs(C_ADI_reshaped.transpose(0, 1, 3, 2)) ** 2, axis=1) / ntraj

    return rho2

In [ ]:
dir_coh="data_coherence/"

In [ ]:
# Compute and save coherence first
#list_methods = [6,8,7,0,3,4,5]
#model_indx = 7
#
#for i, _method_indx in enumerate(list_methods):
#    fn = F"model{model_indx}-method{_method_indx}-icond0/mem_data.hdf"
#    rho2 = compute_coherence(fn)
#
#    np.savetxt(dir_coh+F"coherence_model{model_indx}_method{_method_indx}_21.txt", rho2[:,2,1])

In [ ]:
# Compute and save coherence first
#list_methods = [7]
#model_indx = 7
#
#for i, _method_indx in enumerate(list_methods):
#    fn = F"model{model_indx}-method{_method_indx}-icond0/mem_data.hdf"
#    rho2 = compute_coherence(fn)
#
#    np.savetxt(dir_coh+F"coherence_model{model_indx}_method{_method_indx}_21.txt", rho2[:,2,1])

In [ ]:
def plot_ecp(_model_indx, list_methods = [6,8,7,0,3,4,5], name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"],
             title=None, markers = ["o", "o", "o", "D", "D", "D", "D"], ms= [6, 6, 6, 7, 7, 7, 7], zorder_methods = [11,10,10,0,0,0,0], list_ls=["-", "--", "dotted"], _nstates=2,
             _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    nst = _nstates

    fig, axs = plt.subplots(4, figsize=(10,12), gridspec_kw=dict(hspace=0), dpi=300)
    nr_mark = 30
    
    if title != None: axs[0].set_title(title, fontsize=28)
    
    # energy deviation
    Hart2eV=27.2114
    axs[0].set_ylabel(r"$|\Delta E|$ (mHa)", fontsize=28)
    axs[0].set_xticks([])
    axs[0].tick_params(axis='both', labelsize=28)
    #axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs[0].margins(x=0)
    
    axs[0].plot([0], [0], label='QD', color='k')
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            axs[0].plot(f["time/data"][:]/41.0, np.abs(f["Etot_ave/data"][:]-f["Etot_ave/data"][0])*1000, linewidth=1, 
                label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    if turn_on_legend:
        if _bbox_to_anchor == "":
            axs[0].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper right').set_zorder(100)
        else:
            axs[0].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper right', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    
    # Ehrenfest position
    #axs[1].set_ylabel("position (a.u.)", fontsize=28)
    axs[1].set_ylabel(r"$\langle R \rangle$ (Bohr)", fontsize=28)
    axs[1].set_yticks([-10,10,30])
    axs[1].set_xticks([])
    axs[1].tick_params(axis='both', labelsize=28)
    axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[1].margins(x=0)

    with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        axs[1].plot(f["time/data"][:]/41.0, f["q_adi/data"][:,0], linewidth=7, color="black", zorder=-10)
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            axs[1].plot(f["time/data"][:]/41.0, np.average(f["q/data"][:,:,0], axis=1), 
            linewidth=1, label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i]
            , zorder=zorder_methods[i])
    
    # coherences
    axs[2].set_ylabel("coherence", fontsize=28)
    axs[2].tick_params(axis='both', labelsize=28)
    axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[2].margins(x=0)
    axs[2].set_xticks([])
    with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
    
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                axs[2].plot(f["time/data"][:]/41.0, f["coherence_adi/data"][:, ist, jst], 
                            label=temp, linewidth=7, ls=list_ls[cnt], color = 'black', zorder=-10)
                temp = "_nolegend_"
        
        times = f["time/data"][:]
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
        
        #rho2 = compute_coherence(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf")
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_{ist}{jst}.txt")
                #axs[2].plot(times/41.0, rho2[:, ist, jst], linewidth=1, ls=list_ls[cnt],
                axs[2].plot(times/41.0, rho2, linewidth=1, ls=list_ls[cnt],
                    color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    # populations
    axs[3].set_xlabel("time (fs)", fontsize=28)
    axs[3].set_ylabel("population", fontsize=28)
    axs[3].margins(x=0)
    axs[3].tick_params(axis='both', labelsize=28)
    #axs[3].set_yticks([0.0, 0.5, 1.0])
    
    with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
        #for ist in range(nst):
        #    axs[3].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, ist, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
        axs[3].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, 1, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            
            #for ist in range(nst):
            #    axs[3].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, ist], linewidth=1, 
            #        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
            axs[3].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, 1], linewidth=1, 
                color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

    #fig.tight_layout(pad=_pad)
    fig.align_labels()
    fig.tight_layout()
    #axs[1].legend(fontsize=20)


In [ ]:
with h5py.File(F"model7-method7-icond0/mem_data.hdf", 'r') as f:
    print(np.abs(f["Etot_ave/data"][:]-f["Etot_ave/data"][0])*1000)

with h5py.File(F"model4-method7-icond0/mem_data.hdf", 'r') as f:
    print(np.abs(f["Etot_ave/data"][:]-f["Etot_ave/data"][0])*1000)
    
with h5py.File(F"model5-method7-icond0/mem_data.hdf", 'r') as f:
    print(np.abs(f["Etot_ave/data"][:]-f["Etot_ave/data"][0])*1000)

In [ ]:
# ECR
#plot_ecp(4, _bbox_to_anchor=(-0.01,1.04))
plot_ecp(4, title="ECWR")

In [ ]:
# DAG
plot_ecp(5, title="DAG", _bbox_to_anchor=(-0.01,1.04), turn_on_legend=False)

In [ ]:
# Holstein
plot_ecp(1, title="SC Holstein", _bbox_to_anchor=(-0.01,1.04))

In [ ]:
# Holstein
plot_ecp(3, title="DC Holstein", _bbox_to_anchor=(-0.01,1.04), turn_on_legend=False)

In [ ]:
# Esch-Levine
_model_indx = 7
list_methods = [6,8,7,0,3,4,5]
name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"]
markers = ["o", "o", "o", "D", "D", "D", "D"]
ms = [6, 6, 6, 7, 7, 7, 7]
zorder_methods = [11,10,10,0,0,0,0]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(8, figsize=(11,17), gridspec_kw={'hspace': 0}, dpi=300)
# energy deviation
Hart2eV=27.2114
axs[0].set_ylabel(r'$|\Delta E|$ (mHa)', fontsize=25)
#axs[0].set_ylim([0, 3.5])
axs[0].set_xlim([0, _tlim])
axs[0].set_xticks([])
axs[0].tick_params(axis='both', labelsize=25)
axs[0].margins(x=0)

axs[0].plot([0], [0], label='QD', color='k')
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[0].plot(f["time/data"][:_slim]/41.0, np.abs(f["Etot_ave/data"][:_slim]-f["Etot_ave/data"][0])*1000, linewidth=1,
            label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
        
axs[0].legend(fontsize=25, ncol=4, columnspacing=0.5, loc='center right', frameon=False).set_zorder(100)

# position
#axs[1].set_ylabel("position (a.u.)", fontsize=25)
axs[1].set_ylabel(r"$\langle R \rangle$ (Bohr)", fontsize=25)
axs[1].set_ylim([-5, 39])
axs[1].set_xlim([0, _tlim])
axs[1].set_xticks([])
axs[1].tick_params(axis='both', labelsize=25)
#axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
axs[1].margins(x=0)

with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    axs[1].plot(f["time/data"][:_slim]/41.0, f["q_adi/data"][:_slim,0], linewidth=7, color="black", zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[1].plot(f["time/data"][:_slim]/41.0, np.average(f["q/data"][:_slim,:,0], axis=1), 
        linewidth=1, color = colors[clrs_index[i]],  marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

fn = F"model7-method7-icond0/mem_data.hdf"
with h5py.File(fn, 'r') as f:
    axs[1].plot(f["time/data"][:]/41.0, np.average(f["q/data"][:,:,0], axis=1),
    linewidth=2, linestyle="dashed", color = colors["31"], zorder=5)
        
# coherences, 12
axs[2].set_ylabel(r"$\langle |\rho_{12}|^{2} \rangle $", fontsize=25)
axs[2].tick_params(axis='both', labelsize=25)
axs[2].set_xlim([0, _tlim])
axs[2].margins(x=0)
axs[2].set_xticks([])
axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[2].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 2, 1], label=temp, linewidth=7, color = 'black', zorder=-10)
    
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        times = f["time/data"][:_slim]
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_21.txt")
    #print(times.shape)
    #print(rho2.shape)
    axs[2].plot(times/41.0, rho2[:_slim], linewidth=1,
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# coherences, 01
axs[3].set_ylabel(r"$\langle |\rho_{01}|^{2} \rangle $", fontsize=25)
axs[3].tick_params(axis='both', labelsize=25)
axs[3].set_xlim([0, _tlim])
axs[3].margins(x=0)
axs[3].set_xticks([])
axs[3].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[3].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 1, 0], 
        label=temp, linewidth=7, color = 'black', zorder=-10)
    
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_10.txt")
    axs[3].plot(times/41.0, rho2[:_slim], linewidth=1,
        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# coherences, 02
axs[4].set_ylabel(r"$\langle |\rho_{02}|^{2} \rangle $", fontsize=25)
axs[4].tick_params(axis='both', labelsize=25)
axs[4].set_xlim([0, _tlim])
axs[4].margins(x=0)
axs[4].set_xticks([])
axs[4].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[4].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 2, 0], 
        label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_20.txt")
    axs[4].plot(times/41.0, rho2[:_slim], linewidth=1,
        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
# populations, 2
axs[5].set_ylabel(r"$\langle \rho_{2}\rangle$", fontsize=25)
axs[5].tick_params(axis='both', labelsize=25)
axs[5].set_xlim([0, _tlim])
axs[5].margins(x=0)
axs[5].set_xticks([])
axs[5].set_ylim([0.3, 1.06])
axs[5].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[5].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 2, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[5].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 2], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# populations, 1
axs[6].set_ylabel(r"$\langle \rho_{1}\rangle$", fontsize=25)
axs[6].tick_params(axis='both', labelsize=25)
axs[6].set_xlim([0, _tlim])
axs[6].margins(x=0)
axs[6].set_xticks([])
axs[6].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[6].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 1, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep =_slim
        freq = int(_nstep/nr_mark)
        axs[6].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 1], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# populations, 0
axs[7].set_xlabel("time (fs)", fontsize=25)
axs[7].set_ylabel(r"$\langle \rho_{0}\rangle$", fontsize=25)
axs[7].tick_params(axis='both', labelsize=25)
axs[7].set_xlim([0, _tlim])
axs[7].margins(x=0)
axs[7].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[7].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 0, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[7].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 0], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
#fig.tight_layout(pad=_pad)
fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine / Energy deviation, <R>
_model_indx = 7
list_methods = [6,8,7,0,3,4,5]
name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"]
markers = ["o", "o", "o", "D", "D", "D", "D"]
ms = [6, 6, 6, 7, 7, 7, 7]
zorder_methods = [11,10,10,0,0,0,0]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(2, figsize=(11,10), gridspec_kw={'hspace': 0}, dpi=300)
# energy deviation
Hart2eV=27.2114
axs[0].set_ylabel(r'$|\Delta E|$ (mHa)', fontsize=28)
#axs[0].set_ylim([0, 3.5])
axs[0].set_xlim([0, _tlim])
axs[0].set_xticks([])
axs[0].tick_params(axis='both', labelsize=28)
axs[0].margins(x=0)

axs[0].plot([0], [0], label='QD', color='k')
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[0].plot(f["time/data"][:_slim]/41.0, np.abs(f["Etot_ave/data"][:_slim]-f["Etot_ave/data"][0])*1000, linewidth=1,
            label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
        
axs[0].legend(fontsize=28, ncol=2, columnspacing=0.5, loc='center right', frameon=False).set_zorder(100)

# position
#axs[1].set_ylabel("position (a.u.)", fontsize=25)
axs[1].set_ylabel(r"$\langle R \rangle$ (Bohr)", fontsize=28)
axs[1].set_ylim([-5, 39])
axs[1].set_xlim([0, _tlim])
#axs[1].set_xticks([])
axs[1].set_xticks([0, 50, 100, 150])
axs[1].tick_params(axis='both', labelsize=28)
axs[1].set_xlabel("time (fs)", fontsize=28)
#axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
axs[1].margins(x=0)

with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    axs[1].plot(f["time/data"][:_slim]/41.0, f["q_adi/data"][:_slim,0], linewidth=7, color="black", zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[1].plot(f["time/data"][:_slim]/41.0, np.average(f["q/data"][:_slim,:,0], axis=1), 
        linewidth=1, color = colors[clrs_index[i]],  marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

fn = F"model7-method7-icond0/mem_data.hdf"
with h5py.File(fn, 'r') as f:
    axs[1].plot(f["time/data"][:]/41.0, np.average(f["q/data"][:,:,0], axis=1),
    linewidth=2, linestyle="dashed", color = colors["31"], zorder=5)
        

fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine / Energy deviation, <R>
_model_indx = 7
list_methods = [6,8,7,0,3,4,5]
name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"]
markers = ["o", "o", "o", "D", "D", "D", "D"]
ms = [6, 6, 6, 7, 7, 7, 7]
zorder_methods = [11,10,10,0,0,0,0]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(figsize=(11,8), gridspec_kw={'hspace': 0}, dpi=300)
# energy deviation
Hart2eV=27.2114
axs.set_ylabel(r'$|\Delta E|$ (mHa)', fontsize=30)
#axs.set_ylim([0, 3.5])
axs.set_xlabel("time (fs)", fontsize=30)
axs.set_xlim([0, _tlim])
#axs.set_xticks([])
axs.tick_params(axis='both', labelsize=30)
axs.margins(x=0)

axs.plot([0], [0], label='QD', color='k')
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs.plot(f["time/data"][:_slim]/41.0, np.abs(f["Etot_ave/data"][:_slim]-f["Etot_ave/data"][0])*1000, linewidth=1,
            label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
        
axs.legend(fontsize=30, ncol=1, columnspacing=0.5, loc='center right', bbox_to_anchor=(1.0, 0.46), frameon=False).set_zorder(100)

fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine / Energy deviation, <R>
_model_indx = 7
list_methods = [6,8,7,0,3,4,5]
name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"]
markers = ["o", "o", "o", "D", "D", "D", "D"]
ms = [6, 6, 6, 7, 7, 7, 7]
zorder_methods = [11,10,10,0,0,0,0]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(figsize=(11,8), gridspec_kw={'hspace': 0}, dpi=300)

# position
#axs.set_ylabel("position (a.u.)", fontsize=25)
axs.set_ylabel(r"$\langle R \rangle$ (Bohr)", fontsize=30)
axs.set_ylim([-2, 34.9])
axs.set_yticks([0, 10, 20, 30])
axs.set_xlim([0, _tlim])
#axs.set_xticks([])
axs.set_xticks([0, 50, 100, 150])
axs.tick_params(axis='both', labelsize=30)
axs.set_xlabel("time (fs)", fontsize=30)
#axs.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
axs.margins(x=0)

with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    axs.plot(f["time/data"][:_slim]/41.0, f["q_adi/data"][:_slim,0], linewidth=7, color="black", zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs.plot(f["time/data"][:_slim]/41.0, np.average(f["q/data"][:_slim,:,0], axis=1), 
        linewidth=1, color = colors[clrs_index[i]],  marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

fn = F"model7-method7-icond0/mem_data.hdf"
with h5py.File(fn, 'r') as f:
    axs.plot(f["time/data"][:]/41.0, np.average(f["q/data"][:,:,0], axis=1),
    linewidth=2, linestyle="dashed", color = colors["31"], zorder=5)
        

fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine
_model_indx = 7
list_methods = [6,8,7,0,3,4,5]
name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"]
markers = ["o", "o", "o", "D", "D", "D", "D"]
ms = [6, 6, 6, 7, 7, 7, 7]
zorder_methods = [11,10,10,0,0,0,0]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(3, figsize=(11,10), gridspec_kw={'hspace': 0}, dpi=300)
        
# coherences, 12
axs[0].set_ylabel(r"$\langle |\rho_{12}|^{2} \rangle $", fontsize=28)
axs[0].tick_params(axis='both', labelsize=28)
axs[0].set_xlim([0, _tlim])
axs[0].margins(x=0)
axs[0].set_xticks([])
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[0].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 2, 1], label=temp, linewidth=7, color = 'black', zorder=-10)
    
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        times = f["time/data"][:_slim]
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_21.txt")
    #print(times.shape)
    #print(rho2.shape)
    axs[0].plot(times/41.0, rho2[:_slim], linewidth=1,
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# coherences, 01
axs[1].set_ylabel(r"$\langle |\rho_{01}|^{2} \rangle $", fontsize=28)
axs[1].tick_params(axis='both', labelsize=28)
axs[1].set_xlim([0, _tlim])
axs[1].margins(x=0)
axs[1].set_xticks([])
axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[1].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 1, 0], 
        label=temp, linewidth=7, color = 'black', zorder=-10)
    
for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_10.txt")
    axs[1].plot(times/41.0, rho2[:_slim], linewidth=1,
        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# coherences, 02
axs[2].set_ylabel(r"$\langle |\rho_{02}|^{2} \rangle $", fontsize=28)
axs[2].tick_params(axis='both', labelsize=28)
axs[2].set_xlim([0, _tlim])
axs[2].margins(x=0)
#axs[2].set_xticks([])
axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[2].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 2, 0], 
        label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_20.txt")
    axs[2].plot(times/41.0, rho2[:_slim], linewidth=1,
        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
axs[2].set_xlabel("time (fs)", fontsize=28)

fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine
_model_indx = 7
list_methods = [6,8,7,0,3,4,5]
name_methods = ["SHXF", "MFXF", "MQCXF", "MF", "FSSH", "SDM", "BCSH"]
markers = ["o", "o", "o", "D", "D", "D", "D"]
ms = [6, 6, 6, 7, 7, 7, 7]
zorder_methods = [11,10,10,0,0,0,0]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(3, figsize=(11,10), gridspec_kw={'hspace': 0}, dpi=300)

# populations, 2
axs[0].plot([],[],color="k", label="QD")

axs[0].set_ylabel(r"$\langle \rho_{2}\rangle$", fontsize=28)
axs[0].tick_params(axis='both', labelsize=28)
axs[0].set_xlim([0, _tlim])
axs[0].margins(x=0)
axs[0].set_xticks([])
axs[0].set_ylim([0.3, 1.06])
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[0].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 2, 0], label="", linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[0].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 2], linewidth=1, label=name_methods[i],
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

axs[0].legend(fontsize=26, ncol=4, columnspacing=0.5, loc='upper right', frameon=False).set_zorder(100)
        
# populations, 1
axs[1].set_ylabel(r"$\langle \rho_{1}\rangle$", fontsize=28)
axs[1].tick_params(axis='both', labelsize=28)
axs[1].set_xlim([0, _tlim])
axs[1].margins(x=0)
axs[1].set_xticks([])
axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[1].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 1, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep =_slim
        freq = int(_nstep/nr_mark)
        axs[1].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 1], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# populations, 0
axs[2].set_xlabel("time (fs)", fontsize=28)
axs[2].set_ylabel(r"$\langle \rho_{0}\rangle$", fontsize=28)
axs[2].tick_params(axis='both', labelsize=28)
axs[2].set_xticks([0, 50, 100, 150])
axs[2].set_xlim([0, _tlim])
axs[2].margins(x=0)
axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[2].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 0, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[2].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 0], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
#fig.tight_layout(pad=_pad)
fig.align_labels()
fig.tight_layout()